# TensorFlow显示多项式回归
上两篇文章中我们讲解了单变量线性回归和多元线性回归的例子，对于非线性的数据分布，用单变量线性回归拟合程度一般，我们来试试多项式回归。

前面的步骤还是和上篇一样，后面会添加多个变量来对数据进行拟合。

In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

1. 数据准备 

In [ ]:
n_observations = 100
xs = np.linspace(-3, 3, n_observations)
ys = np.sin(xs) + np.random.uniform(-0.5, 0.5, n_observations)  # 添加给定伽马分布随机的噪声
plt.scatter(xs, ys, linewidths=0.01)
plt.show()

2 . 准备好placeholder,开好容器来装数据

In [ ]:
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')

3. 定义权值和偏执并初始化

In [ ]:
w = tf.Variable(tf.random_normal([1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

4. 计算预测结果

In [ ]:
Y_pred = tf.add(tf.multiply(X, w), b)  # tf.multiply() 对应元素相乘

#添加高次项
w_2 = tf.Variable(tf.random_normal([1]), name='weight_2')
Y_pred = tf.add(tf.multiply(tf.pow(X, 2), w_2), Y_pred)

w_3 = tf.Variable(tf.random_normal([1]), name='weight_3')
Y_pred = tf.add(tf.multiply(tf.pow(X, 3), w_3), Y_pred)

# W_4 = tf.Variable(tf.random_normal([1]),name = 'weight_4')
# Y_pred = tf.add(tf.multiply(tf.pow(X,4),W_4), Y_pred)

5. 计算损失值

In [ ]:
sample_num = xs.shape[0]  # 取出xs的个数，这个就是前面定义的 n_observations=100
loss = tf.reduce_sum(tf.pow(Y_pred - Y, 2)) / sample_num  #向量对应的点相减之后，求平方和，在除以点的个数 （均方差损失函数）

6. 初始化optimizer
梯度下降函数保持不变，使得loss函数的值取到极小值

In [ ]:
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

7. 指定迭代次数，并在 session 执行 graph
    
    ```python
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    ```
    
    再次强调！！！此处两句极其重要，需要先初始化变量，才能进行读取和写入（赋值）操作

In [ ]:
n_sample = xs.shape[0]
loss_total=[]
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)

    for i in range(500):
        total_loss = 0
        for x, y in zip(xs, ys):
            _, l = sess.run([optimizer, loss], feed_dict={X: x, Y: y})
            total_loss += l
        loss_total.append(total_loss / n_sample)
        if (i + 1) % 100 == 0:
            print("Epoch:{0} loss:{1} W:{2} W_2:{3} W_3:{4} B:{5}".format(
                i + 1, total_loss / n_sample, w.eval(), w_2.eval(), w_3.eval(),
                b.eval()))
    W, W_2, W_3, B = sess.run([w, w_2, w_3, b])

In [ ]:
plt.plot(loss_total)
plt.show()
plt.scatter(xs, ys)
plt.plot(xs, xs**3 * W_3 + xs**2 * W_2 + xs * W + B, 'r')
plt.show()